# **Preprocesamiento de texto NLP USANDO LIBRERIAS**

**By Jean Carlo Alvarez Ramirez**

En la demo vamos a ver:

- Limpieza, la remoción del contenido no deseado

- Normalización, la conversión diferentes formas a una sola

- Tokenización, la separación del texto en tókenes (unidades mínimas, por ejemplo palabras)

- Separación en conjuntos de datos: entrenamiento, validación, prueba

- Generación del vocabulario, la lista de tókenes conocidos

- Creacion de embeddings



# Importar librerias

In [50]:
import re
import string
import random
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import movie_reviews
from gensim.models import Word2Vec

**import re:** Importa el módulo re para trabajar con expresiones regulares, útil para buscar, reemplazar o filtrar texto.

**import string:** Importa el módulo string que proporciona constantes y funciones para manipular cadenas, como listas de caracteres especiales o letras.

**import random:** Importa el módulo random para generar números aleatorios, mezclar listas o realizar selecciones aleatorias.

**import numpy as np:** Importa NumPy como np, una biblioteca para operaciones matemáticas y manipulación de arreglos multidimensionales.

**from sklearn.model_selection import train_test_split:** Importa train_test_split de scikit-learn, que divide los datos en conjuntos de entrenamiento y prueba.

**Parámetros importantes:**
- test_size: Proporción de datos para el conjunto de prueba.
- random_state: Fija la semilla para reproducibilidad.

**import nltk:** Importa NLTK (Natural Language Toolkit), una biblioteca para procesamiento de lenguaje natural.

**from nltk.tokenize import word_tokenize**: Importa word_tokenize para dividir texto en palabras (tokens).

**from nltk.stem import WordNetLemmatizer:** Importa WordNetLemmatizer para reducir palabras a su forma base (ej: "running" → "run").

**from nltk import pos_tag:** Importa pos_tag para etiquetar palabras con su categoría gramatical (sustantivo, verbo, etc.).

**from nltk.corpus import movie_reviews:** Importa el corpus movie_reviews de NLTK, que contiene reseñas de películas etiquetadas como positivas o negativas.

**from gensim.models import Word2Vec:** Importa Word2Vec de la biblioteca Gensim para crear modelos de vectores de palabras basados en su contexto.

**Parámetros clave:**
- vector_size: Tamaño del vector de cada palabra.
- window: Número de palabras de contexto.
- min_count: Palabras con frecuencia menor se descartan.
- sg: Si es 1 usa Skip-Gram, si es 0 usa CBOW.

**Parámetro sg en Word2Vec:**
El parámetro sg en el modelo Word2Vec determina qué arquitectura utilizar para entrenar los vectores de palabras:

- sg=0 → CBOW (Continuous Bag of Words)
- sg=1 → Skip-Gram
**¿Qué es CBOW (Continuous Bag of Words)?**

Funcionamiento: Predice la palabra objetivo usando las palabras de contexto cercanas

Ejemplo: Si tienes la frase:

"El gato está sobre la mesa",
y la palabra objetivo es "está", el modelo usa las palabras cercanas ("El", "gato", "sobre", "la") para predecir "está"

Uso recomendado:
- Con datasets grandes
- Cuando se quiere mayor rapidez en el entrenamiento
- Mejora la precisión en palabras frecuentes

**¿Qué es Skip-Gram?**

Funcionamiento: Hace lo contrario a CBOW. Usa la palabra objetivo para predecir su contexto

Ejemplo: Con la misma frase y palabra objetivo "está", el modelo intenta predecir las palabras cercanas ("El", "gato", "sobre", "la")

Uso recomendado:
- Con datasets pequeños.
- Mejora el rendimiento con palabras raras o poco frecuentes
- Captura mejor las relaciones semánticas complejas

- sg=0 (CBOW): El modelo entrenará más rápido y funcionará mejor en corpus grandes y palabras comunes

- sg=1 (Skip-Gram): El entrenamiento será más lento, pero detectará mejor relaciones entre palabras raras o específicas


# Descargar recursos necesarios de NLTK

In [51]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('movie_reviews')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

**nltk.download('punkt')**

¿Qué hace? Descarga el tokenizador Punkt, que segmenta texto en oraciones y palabras

Uso: Necesario para funciones como word_tokenize()

**nltk.download('wordnet')**

¿Qué hace? Descarga la base de datos WordNet, una red semántica de palabras en inglés

Uso: Esencial para lematización con WordNetLemmatizer() y obtener sinónimos, antónimos o jerarquías léxicas

**nltk.download('omw-1.4')**

¿Qué hace? Descarga Open Multilingual WordNet, que amplía WordNet con traducciones a varios idiomas

Uso: Permite lematización y análisis semántico en otros idiomas además del inglés

**nltk.download('averaged_perceptron_tagger')**

¿Qué hace? Descarga el etiquetador Averaged Perceptron Tagger, usado para análisis gramatical (Part-of-Speech tagging)

Uso: Funciona con pos_tag() para asignar etiquetas gramaticales (sustantivo, verbo, adjetivo, etc.) a las palabras

**nltk.download('movie_reviews')**

¿Qué hace? Descarga el corpus de reseñas de películas de NLTK, con reseñas etiquetadas como positivas o negativas

Uso: Popular para entrenar y probar modelos de análisis de sentimientos

# Cargar Dataset

Cargamos el dataset usando el corpus de reseñas de películas

La función cargar_dataset obtiene un número limitado de reseñas del corpus movie_reviews de NLTK. En este ejemplo se usan 10 reseñas para simplificar la demostración

In [52]:
def cargar_dataset(num_reviews=10):
    """
    Carga un número limitado de reseñas para el ejemplo.
    """
    fileids = movie_reviews.fileids()[:num_reviews]
    textos = [movie_reviews.raw(fileid) for fileid in fileids]
    return textos

**def cargar_dataset(num_reviews=10):**

Define la función cargar_dataset con un parámetro num_reviews, que por defecto es 10

- num_reviews controla cuántas reseñas se cargan

**fileids = movie_reviews.fileids()[:num_reviews]**

- movie_reviews.fileids() obtiene todos los identificadores de archivos del corpus movie_reviews
- [:num_reviews] selecciona los primeros num_reviews identificadores

**textos = [movie_reviews.raw(fileid) for fileid in fileids]**

Usa una lista para iterar sobre los fileids

- movie_reviews.raw(fileid) carga el texto completo de cada reseña

Crea una lista llamada textos que contiene los textos cargados.


# Limpieza del texto

Conversión a minúsculas, eliminación de dígitos y puntuación

La función limpiar_texto transforma el texto a minúsculas, elimina dígitos y signos de puntuación, y recorta espacios adicionales, dejando un texto listo para la tokenizanizar

In [53]:
def limpiar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r'\d+', '', texto)
    texto = re.sub(r'[' + re.escape(string.punctuation) + ']', '', texto)
    texto = texto.strip()
    return texto

**def limpiar_texto(texto):**

Define la función limpiar_texto que toma como parámetro un string texto

**texto = texto.lower()**

Convierte todo el texto a minúsculas

Ejemplo: "Hello World!" → "hello world!"

**texto = re.sub(r'\d+', '', texto)**

Usa expresiones regulares para eliminar todos los números del texto

- r'\d+' busca secuencias de uno o más dígitos

Ejemplo: "Movie 123" → "Movie "

**texto = re.sub(r'[' + re.escape(string.punctuation) + ']', '', texto)**

Elimina los signos de puntuación utilizando **re.escape(string.punctuation)** para identificar caracteres especiales

Ejemplo: "Hello, world!" → "Hello world"

**texto = texto.strip()**

Elimina espacios en blanco al inicio y al final del texto

Ejemplo: " hello world " → "hello world"

Que pasaria si cambiamos:

- Eliminas texto.lower() → Mantendrás las mayúsculas
- Cambias r'\d+' por r'\d' → Eliminarías cada dígito individualmente
- Quitas re.escape(string.punctuation) → No eliminarías los signos de puntuación

# Tokenizacion

Se utiliza word_tokenize de NLTK para separar el texto en tokens (palabras y otros símbolos)

In [54]:
def tokenizar_texto(texto):
    """
    Separamos el texto en tokens usando word_tokenize de NLTK.
    """
    tokens = word_tokenize(texto)
    return tokens

**def tokenizar_texto(texto):**

Define la función tokenizar_texto que recibe un string texto como entrada

**tokens = word_tokenize(texto)**

Utiliza la función word_tokenize() de NLTK para dividir el texto en tokens (palabras, signos de puntuación, números, etc.)

word_tokenize usa el modelo Punkt el cual  descargasmos con nltk.download('punkt')) en un inicio

**Tener en cuenta?**
- Incluye signos de puntuación: Si necesitas eliminarlos, debes filtrarlos después

- Respeta espacios y caracteres especiales: Cada palabra y símbolo se considera un token separado

- Soporta varios idiomas, aunque funciona mejor con inglés por defecto

# Normalizacion

Lematización con POS-tagging

La función get_wordnet_pos mapea las etiquetas de parte del habla (POS) generadas por pos_tag de NLTK al formato esperado por WordNetLemmatizer.



In [55]:
def get_wordnet_pos(nltk_pos_tag):
    """
    Convierte la etiqueta POS de NLTK a un formato compatible con WordNetLemmatizer.
    """
    if nltk_pos_tag.startswith('J'):
        return 'a'  # adjetivo
    elif nltk_pos_tag.startswith('V'):
        return 'v'  # verbo
    elif nltk_pos_tag.startswith('N'):
        return 'n'  # sustantivo
    elif nltk_pos_tag.startswith('R'):
        return 'r'  # adverbio
    else:
        return 'n'  # por defecto, sustantivo

**def get_wordnet_pos(nltk_pos_tag):**

Define la función get_wordnet_pos que recibe nltk_pos_tag, la etiqueta gramatical generada por pos_tag() de NLTK

**if nltk_pos_tag.startswith('J'):**

Si la etiqueta empieza con 'J' (ej. 'JJ' para adjetivos), devuelve 'a' (adjetivo) para WordNet

**elif nltk_pos_tag.startswith('V'):**

Si la etiqueta empieza con 'V' (ej. 'VB', 'VBD', 'VBG'), devuelve 'v' (verbo)

**elif nltk_pos_tag.startswith('N'):**

Si la etiqueta empieza con 'N' (ej. 'NN' para sustantivos), devuelve 'n' (sustantivo)

**elif nltk_pos_tag.startswith('R'):**

Si la etiqueta empieza con 'R' (ej. 'RB' para adverbios), devuelve 'r' (adverbio)

**else: return 'n'**

Si la etiqueta no coincide con las anteriores, asigna por defecto 'n' (sustantivo)

**Por qué es necesaria esta función?**

- WordNetLemmatizer necesita etiquetas específicas ('a', 'v', 'n', 'r') para realizar una lematización precisa

- Las etiquetas de pos_tag() siguen el estándar Penn Treebank (ej.: 'JJ', 'VB', 'NN', 'RB'), por lo que se requiere esta conversión

**Qué pasa si no uso esta función?**

El lematizador asume que todas las palabras son sustantivos

Ejemplo:
- Sin conversión: "running" → "running"
- Con conversión ('V'): "running" → "run"

En WordNetLemmatizer se usan específicamente las etiquetas 'a', 'v', 'n' y 'r' porque son los tipos de palabras que WordNet reconoce para la lematización. Cualquier otra etiqueta no será interpretada correctamente por el lematizador.

**Etiquetas válidas para WordNetLemmatizer:**

**Etiqueta	Categoría Gramatical**

**Ejemplo**
- 'n'	Noun (Sustantivo)	dog → dog

- 'v'	Verb (Verbo)	running → run

- 'a'	Adjective (Adjetivo)	better → good

- 'r'	Adverb (Adverbio)	quickly → quick

- 's'	Adjective Satellite (Raro)	good-looking → good

Aunque 's' existe en WordNet, la mayoría de lematizadores lo tratan como 'a'

**Qué pasa si uso otras letras?**

Si pasas una letra no válida (ej. 'P' para pronombres o 'C' para conjunciones), WordNetLemmatizer no sabrá cómo tratarla

En ese caso:
- No se aplica la lematización y la palabra queda igual
- O se toma la categoría por defecto ('n', sustantivo).

In [56]:
def normalizar_tokens(tokens):
    """
    Lematiza cada token utilizando su parte del habla obtenida con pos_tag
    """
    lemmatizer = WordNetLemmatizer()
    pos_tokens = pos_tag(tokens)
    lemmatized_tokens = [
        lemmatizer.lemmatize(token, pos=get_wordnet_pos(tag)) for token, tag in pos_tokens
    ]
    return lemmatized_tokens

normalizar_tokens lematiza cada token utilizando el POS correspondiente, reduciendo las palabras a su forma base

**def normalizar_tokens(tokens):**

Define la función que toma como entrada una lista de tokens (palabras individuales)

**lemmatizer = WordNetLemmatizer()**

Crea una instancia del lematizador de WordNet

Este se usa para reducir palabras a su forma base (lematización)

**pos_tokens = pos_tag(tokens)**

Usa pos_tag() de NLTK para asignar etiquetas gramaticales a cada token
Ejemplo:
- Codigo : pos_tag(['running', 'fast'])
- Salida: [('running', 'VBG'), ('fast', 'RB')]

**lemmatized_tokens = [lemmatizer.lemmatize(token, pos=get_wordnet_pos(tag)) for token, tag in pos_tokens]**

Usa una list comprehension para lematizar cada token

- get_wordnet_pos(tag) convierte la etiqueta de NLTK (ej. 'VBG') al formato compatible con WordNet (ej. 'v')

- lemmatizer.lemmatize() aplica la lematización usando la categoría correcta

**Qué hacemos exactamente en esta funcion?**

Identifica el rol gramatical de cada palabra (sustantivo, verbo, adjetivo, adverbio)

Lematiza basándose en ese rol:

- "hanging" (verbo) → "hang"
- "feet" (sustantivo) → "foot"
- "best" (adjetivo) → "good"



# Gnerar Vocabulario

Generamos vocabulario a partir de los tokens del conjunto de entrenamiento

Se recorre el conjunto de entrenamiento para contar la frecuencia de cada token y asignar un identificador único. Se incluyen tokens especiales (PAD y UNK) para facilitar futuros procesos

In [57]:
def generar_vocabulario(tokenized_texts, min_freq=1):
    token_freq = {}
    for tokens in tokenized_texts:
        for token in tokens:
            token_freq[token] = token_freq.get(token, 0) + 1

    # Se agregan tokens especiales
    vocab = {"<PAD>": 0, "<UNK>": 1}
    next_id = 2
    for token, freq in token_freq.items():
        if freq >= min_freq:
            vocab[token] = next_id
            next_id += 1
    return vocab

**def generar_vocabulario(tokenized_texts, min_freq=1):**

Define la función que genera un vocabulario a partir de textos tokenizados

**Parámetros:**
- tokenized_texts: Lista de listas de tokens
- min_freq: Frecuencia mínima para incluir un token en el vocabulario (por defecto, 1)

**token_freq = {}**

Crea un diccionario vacío para contar la frecuencia de cada token

**for tokens in tokenized_texts:**

Itera sobre cada lista de tokens en los textos

**for token in tokens:**

Itera sobre cada token individual

**token_freq[token] = token_freq.get(token, 0) + 1**

Cuenta cuántas veces aparece cada token

- get(token, 0) devuelve 0 si el token no está aún en el diccionario

**vocab = {"PAD": 0, "UNK": 1}**

Inicializa el vocabulario con dos tokens especiales:

- PAD (ID: 0) → Para rellenar secuencias cortas hasta igualarlas en longitud

- UNK (ID: 1) → Para palabras desconocidas o fuera de vocabulario

**next_id = 2**

Comienza a asignar IDs desde 2 para los tokens reales

**for token, freq in token_freq.items():**

Itera sobre los tokens y sus frecuencias

**if freq >= min_freq:**

Solo incluye tokens que aparecen al menos min_freq veces

**vocab[token] = next_id**

Asigna un ID único al token y lo agrega al vocabulario

**next_id += 1**

Incrementa el ID para el siguiente token

**Que pasa si cambias min_freq?**
- min_freq=1 → Incluye todos los tokens que aparecen al menos una vez
- min_freq=2 → Solo incluye tokens que aparecen al menos dos veces

**Por qué usar PAD y UNK?**
- PAD : Para crear lotes de datos uniformes en tamaño al entrenar modelos
- UNK : Maneja palabras no vistas durante el entrenamiento


# Creacion de Embeddings

Usmos Word2Vec

Se entrena un modelo Word2Vec con los textos de entrenamiento (ya tokenizados y lematizados) para generar representaciones vectoriales  de cada token. Para cada token del vocabulario, si se encuentra en el modelo, se asigna su vector; en caso contrario, se asigna un vector de ceros

In [58]:
def crear_embeddings(train_texts, vocab, embedding_dim=50):
    """
    Entrena un modelo Word2Vec con los textos de entrenamiento y genera una representación
    vectorial para cada token del vocabulario
    """
    # Entrenar Word2Vec usando los textos (tokenizados y lematizados)
    w2v_model = Word2Vec(
        sentences=train_texts,
        vector_size=embedding_dim,
        window=5,
        min_count=1,
        workers=4,
        seed=42
    )

    # Crear un diccionario de embeddings: si el token no se encuentra, se asigna un vector de ceros.
    embeddings = {}
    for token in vocab.keys():
        if token in w2v_model.wv:
            embeddings[token] = w2v_model.wv[token]
        else:
            embeddings[token] = np.zeros(embedding_dim)
    return embeddings, w2v_model

**def crear_embeddings(train_texts, vocab, embedding_dim=50):**

Define la función que genera vectores de palabras (embeddings) usando Word2Vec

**Parámetros:**
- train_texts: Lista de textos tokenizados para entrenar el modelo
- vocab: Diccionario de vocabulario generado previamente
- embedding_dim: Dimensión de los vectores de palabras (por defecto, 50)

**Entrenamiento del modelo Word2Vec:**

**w2v_model = Word2Vec(**

Entrena un modelo Word2Vec con los textos proporcionados

**Parámetros:**

- sentences=train_texts: Los textos tokenizados usados para el entrenamiento
- vector_size=embedding_dim: Dimensión de los vectores generados (50 por defecto)
- window=5: Número de palabras de contexto consideradas antes y después de la palabra objetivo
- min_count=1: Incluye todas las palabras que aparecen al menos 1 vez
- workers=4: Usa 4 núcleos para paralelizar el entrenamiento
- seed=42: Fija la semilla para resultados reproducibles

**Generación del diccionario de embeddings:**

**embeddings = {}**

Crea un diccionario vacío para almacenar los vectores de cada token

**for token in vocab.keys():**

Itera sobre todos los tokens del vocabulario

**if token in w2v_model.wv:**

Verifica si el token tiene un vector entrenado en el modelo

**embeddings[token] = w2v_model.wv[token]**

Si el token está en el modelo, asigna su vector al diccionario

**else: embeddings[token] = np.zeros(embedding_dim)**

Si el token no tiene vector (poco frecuente o desconocido), asigna un vector de ceros

**return embeddings, w2v_model**

Devuelve:
- embeddings : Diccionario de vectores para cada token.
- w2v_model : El modelo entrenado (para futuros usos como similitud de palabras)

**Que pasa si cambias los parámetros?**
- embedding_dim: Aumentarlo puede capturar más información semántica, pero requiere más memoria

- window: Valores grandes amplían el contexto, pero pueden diluir el enfoque en palabras cercanas

- min_count: Si lo subes (ej. min_count=2), se ignorarán palabras poco frecuentes

# Ejecutamos el preprocesamiento

In [65]:

# Cargar dataset (se utilizan 10 reseñas para el ejemplo)
textos = cargar_dataset(num_reviews=10)

# Pre-procesamiento:
# Limpieza del texto
textos_limpiados = [limpiar_texto(texto) for texto in textos]

# Tokenización utilizando NLTK
tokenized_texts = [tokenizar_texto(texto) for texto in textos_limpiados]

# Normalización: Lematización de los tokens
normalized_texts = [normalizar_tokens(tokens) for tokens in tokenized_texts]

# División en conjuntos: Entrenamiento, Validación y Prueba
train_texts, temp_texts = train_test_split(normalized_texts, test_size=0.4, random_state=42)
val_texts, test_texts = train_test_split(temp_texts, test_size=0.5, random_state=42)

# Generación del vocabulario usando solo el conjunto de entrenamiento
vocab = generar_vocabulario(train_texts, min_freq=1)

# Creación de embeddings usando Word2Vec
embeddings, w2v_model = crear_embeddings(train_texts, vocab, embedding_dim=50)

Ejecutamos todas las funciones creadas

# Resultados

In [67]:
print("Ejemplos de textos originales:")
for t in textos[:2]:
    print("  ", t[:100], "...")

Ejemplos de textos originales:
   plot : two teen couples go to a church party , drink and then drive . 
they get into an accident . 
 ...
   the happy bastard's quick movie review 
damn that y2k bug . 
it's got a head start in this movie sta ...


In [68]:
print("\nTextos Limpiados, Tokenizados y Lematizados:")
for tokens in normalized_texts[:2]:
    print("  ", tokens)


Textos Limpiados, Tokenizados y Lematizados:
   ['plot', 'two', 'teen', 'couple', 'go', 'to', 'a', 'church', 'party', 'drink', 'and', 'then', 'drive', 'they', 'get', 'into', 'an', 'accident', 'one', 'of', 'the', 'guys', 'dy', 'but', 'his', 'girlfriend', 'continue', 'to', 'see', 'him', 'in', 'her', 'life', 'and', 'have', 'nightmare', 'whats', 'the', 'deal', 'watch', 'the', 'movie', 'and', 'sorta', 'find', 'out', 'critique', 'a', 'mindfuck', 'movie', 'for', 'the', 'teen', 'generation', 'that', 'touch', 'on', 'a', 'very', 'cool', 'idea', 'but', 'present', 'it', 'in', 'a', 'very', 'bad', 'package', 'which', 'be', 'what', 'make', 'this', 'review', 'an', 'even', 'hard', 'one', 'to', 'write', 'since', 'i', 'generally', 'applaud', 'film', 'which', 'attempt', 'to', 'break', 'the', 'mold', 'mess', 'with', 'your', 'head', 'and', 'such', 'lose', 'highway', 'memento', 'but', 'there', 'be', 'good', 'and', 'bad', 'way', 'of', 'make', 'all', 'type', 'of', 'film', 'and', 'these', 'folk', 'just', 'didn

In [70]:
print("\nVocabulario Generado:")
print(vocab)


Vocabulario Generado:
{'<PAD>': 0, '<UNK>': 1, 'thats': 2, 'exactly': 3, 'how': 4, 'long': 5, 'the': 6, 'movie': 7, 'felt': 8, 'to': 9, 'me': 10, 'there': 11, 'werent': 12, 'even': 13, 'nine': 14, 'laugh': 15, 'in': 16, 'month': 17, 'it': 18, 'a': 19, 'terrible': 20, 'mess': 21, 'of': 22, 'star': 23, 'man': 24, 'mr': 25, 'hugh': 26, 'grant': 27, 'huge': 28, 'dork': 29, 'not': 30, 'whole': 31, 'oralsexprostitution': 32, 'thing': 33, 'refer': 34, 'that': 35, 'bug': 36, 'fact': 37, 'be': 38, 'annoy': 39, 'just': 40, 'adam': 41, 'sandlerannoying': 42, 'talk': 43, 'jim': 44, 'carreyannoying': 45, 'since': 46, 'when': 47, 'do': 48, 'eye': 49, 'flutter': 50, 'and': 51, 'nervous': 52, 'smile': 53, 'pas': 54, 'for': 55, 'act': 56, 'but': 57, 'on': 58, 'other': 59, 'hand': 60, 'really': 61, 'bad': 62, 'slapstick': 63, 'fistfight': 64, 'delivery': 65, 'room': 66, 'culminating': 67, 'head': 68, 'joan': 69, 'cusacks': 70, 'lapa': 71, 'scene': 72, 'he': 73, 'pay': 74, 'have': 75, 'include': 76, 'ob

In [73]:
# Ejemplo: Mostrar el embedding de la palabra "girl" (si se encuentra en el vocabulario)
palabra = "girl"
if palabra in embeddings:
    print(f"\nEmbedding para la palabra '{palabra}':")
    print(embeddings[palabra])
else:
    print(f"\nLa palabra '{palabra}' no se encuentra en el vocabulario.")


Embedding para la palabra 'girl':
[ 1.03391642e-02 -5.36207156e-03  4.44330089e-03  2.00380534e-02
 -1.30567346e-02 -1.52889667e-02 -1.13486992e-02 -1.31497337e-02
  1.25151183e-02  1.52393887e-02  1.96565664e-03 -1.40704457e-02
  1.37699293e-02  1.76588986e-02 -3.13013862e-03  6.74422318e-03
  4.96512931e-03  1.17316540e-05 -6.74585719e-03 -9.05813370e-03
 -8.93647224e-03 -5.68999629e-03  1.06144901e-02  1.31454710e-02
 -1.45915430e-02 -1.81185752e-02 -1.15507934e-03  1.07407784e-02
  1.99506455e-03  1.24793760e-02  8.73896480e-03  7.90904928e-03
  9.74643161e-04 -1.74114890e-02 -1.34181445e-02  9.56689473e-03
 -7.43261632e-03  7.38152768e-03 -6.49460964e-03 -1.87230464e-02
  6.76638912e-03  1.63569096e-02 -1.05935556e-03 -1.55380545e-02
 -3.14152497e-03  1.82556584e-02 -6.98845740e-03 -1.60690006e-02
  1.12086842e-02 -1.10441456e-02]
